# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import pprint

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [4]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

631

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [10]:
#---- REQUESTS 10 SAMPLES FOR TESTING. CHANGE TO ALL CITIES FOR FINAL CODE ---

#Store the base url
url = 'https://api.openweathermap.org/data/2.5/weather'

#List to hold the response data
response_list = []

#Iterate through the list of cities
for city in cities:
    
    #Attempt to retrieve city data
    try:
        #Build the query url
        query_url = f'{url}?q={city}&appid={weather_api_key}&units=imperial'
        
        #Print a message to show the city being processed
        print(f"Processing data for city {cities.index(city) + 1}: {city}.")
        
        #Make API request
        response = requests.get(query_url).json()

        #Create a dictionary of the desired data from the response
        response_dict = {'City': response['name'],
                         'Cloudiness': response['clouds']['all'],
                         'Country': response['sys']['country'],
                         'Date': response['dt'],
                         'Humidity': response['main']['humidity'],
                         'Lat': response['coord']['lat'],
                         'Lng': response['coord']['lon'],
                         'Max Temp': response['main']['temp_max'],
                         'Wind Speed': response['wind']['speed']
                        }
        #Add the response dictionary to the list of response data
        response_list.append(response_dict)
    
    #If the city is not found
    except:
        #Print a message and skip processing for this city
        print(f"City {cities.index(city) + 1} not found. Skipping...")

#Print a message when data retrieval is complete
print('''
-------------------------
Data retrieval complete.
-------------------------
''')

Processing data for city 1: morfelden-walldorf.
Processing data for city 2: pevek.
Processing data for city 3: jamestown.
Processing data for city 4: kavaratti.
Processing data for city 5: faanui.
Processing data for city 6: albany.
Processing data for city 7: vila velha.
Processing data for city 8: russell.
Processing data for city 9: mys shmidta.
City 9 not found. Skipping...
Processing data for city 10: airai.
Processing data for city 11: hobart.
Processing data for city 12: mount isa.
Processing data for city 13: nelson bay.
Processing data for city 14: tuktoyaktuk.
Processing data for city 15: arraial do cabo.
Processing data for city 16: moses lake.
Processing data for city 17: rikitea.
Processing data for city 18: stony plain.
Processing data for city 19: avarua.
Processing data for city 20: carnarvon.
Processing data for city 21: valparaiso.
Processing data for city 22: yellowknife.
Processing data for city 23: nikolskoye.
Processing data for city 24: westport.
Processing data 

Processing data for city 193: ola.
Processing data for city 194: odienne.
Processing data for city 195: odweyne.
Processing data for city 196: marrakesh.
Processing data for city 197: lufilufi.
Processing data for city 198: shalinskoye.
Processing data for city 199: banjar.
Processing data for city 200: general roca.
Processing data for city 201: pierre.
Processing data for city 202: tuatapere.
Processing data for city 203: hudiksvall.
Processing data for city 204: avon park.
Processing data for city 205: umzimvubu.
City 205 not found. Skipping...
Processing data for city 206: lieksa.
Processing data for city 207: kaeo.
Processing data for city 208: severnyy.
Processing data for city 209: bambanglipuro.
Processing data for city 210: nanyang.
Processing data for city 211: necochea.
Processing data for city 212: sataua.
City 212 not found. Skipping...
Processing data for city 213: beloha.
Processing data for city 214: luderitz.
Processing data for city 215: norman wells.
Processing data 

City 387 not found. Skipping...
Processing data for city 388: plettenberg bay.
Processing data for city 389: durban.
Processing data for city 390: gurupi.
Processing data for city 391: jieshi.
Processing data for city 392: mchinji.
Processing data for city 393: hohhot.
Processing data for city 394: chardara.
Processing data for city 395: kibaya.
Processing data for city 396: san rafael.
Processing data for city 397: sterling.
Processing data for city 398: litoral del san juan.
City 398 not found. Skipping...
Processing data for city 399: lorengau.
Processing data for city 400: ust-nera.
Processing data for city 401: gat.
Processing data for city 402: indramayu.
Processing data for city 403: erzin.
Processing data for city 404: viligili.
City 404 not found. Skipping...
Processing data for city 405: concepcion.
Processing data for city 406: bac lieu.
Processing data for city 407: claveria.
Processing data for city 408: norfolk.
Processing data for city 409: mahenge.
Processing data for c

Processing data for city 583: we.
City 583 not found. Skipping...
Processing data for city 584: hervey bay.
Processing data for city 585: malakal.
Processing data for city 586: bryant.
Processing data for city 587: kloulklubed.
Processing data for city 588: ensenada.
Processing data for city 589: mitu.
Processing data for city 590: saint-francois.
Processing data for city 591: grand-lahou.
Processing data for city 592: djenne.
Processing data for city 593: mandera.
Processing data for city 594: sokoto.
Processing data for city 595: tazmalt.
Processing data for city 596: iquitos.
Processing data for city 597: fort-shevchenko.
Processing data for city 598: blackpool.
Processing data for city 599: nioro.
Processing data for city 600: acapulco.
Processing data for city 601: broome.
Processing data for city 602: sand springs.
Processing data for city 603: pacific grove.
Processing data for city 604: kenora.
Processing data for city 605: shimoda.
Processing data for city 606: arandas.
Proces

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [11]:
#Export the city data to a .csv file
response_df = pd.DataFrame(response_list)
response_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Mörfelden-Walldorf,0,DE,1586384460,57,49.99,8.58,64.00,2.24
1,Pevek,100,RU,1586384461,92,69.70,170.31,-3.39,5.59
2,Jamestown,1,US,1586384333,66,42.10,-79.24,57.99,13.87
3,Kavaratti,4,IN,1586384462,66,10.57,72.64,83.52,8.77
4,Faanui,69,PF,1586384462,75,-16.48,-151.75,83.16,13.11
...,...,...,...,...,...,...,...,...,...
581,Sept-Îles,40,CA,1586384948,40,50.20,-66.38,37.40,11.41
582,Shangrao,34,CN,1586385041,66,28.45,117.97,63.32,3.13
583,Jumla,1,NP,1586385042,27,29.27,82.18,38.66,5.03
584,Vera Cruz,20,BR,1586385042,78,-12.63,-41.03,80.60,2.24


### Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

#### Latitude vs. Temperature Plot

In [ ]:
# --- NEED TO ADD DATE OF ANALYSIS TO CHART TITLE ---

#Create a scatter plot of temperature vs. latitude for cities in the response data frame
x_values = response_df['Lat']
y_values = response_df['Max Temp']
plt.scatter(x_values, y_values)

#Add plot titles/labels
plt.title('City Latitude vs. Temperature (Date)')
plt.xlabel('Latitude')
plt.ylabel('Temperature (F)')
plt.grid()
plt.show()

#### Latitude vs. Humidity Plot

#### Latitude vs. Cloudiness Plot

#### Latitude vs. Wind Speed Plot

## Linear Regression

In [ ]:
# OPTIONAL: Create a function to create Linear Regression plots


In [ ]:
# Create Northern and Southern Hemisphere DataFrames


####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression